In [1]:
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pandas.plotting import scatter_matrix
from sklearn import cluster
from scipy import stats  ## for z-score. do we still need it?

In [2]:
# Read crime data
crimedata = pd.read_csv('Data_tables_Criminal_Incidents_Visualisation_year_ending_March_2018.csv')
crimedata.head()

,Year ending March,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,646
1,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,129
2,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,601
3,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,108
4,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",142


# Process the data

In [3]:
crimedata['Incidents Recorded'] = crimedata['Incidents Recorded'].str.replace(',', '')
crimedata['Incidents Recorded'] = crimedata['Incidents Recorded'].apply(pd.to_numeric)

crimedata.dtypes

Year ending March       int64
Postcode                int64
Suburb/Town Name       object
Offence Division       object
Offence Subdivision    object
Offence Subgroup       object
Incidents Recorded      int64
dtype: object

In [4]:
# Sort offenses by Offence Division to identify targeted crimes that take place in public

crimedata['Offence Division'].value_counts()

B Property and deception offences       117983
A Crimes against the person              74194
D Public order and security offences     34835
E Justice procedures offences            24523
C Drug offences                          19833
F Other offences                          6385
Name: Offence Division, dtype: int64

In [5]:
mask_public = crimedata.isin({'Offence Division': ['A Crimes against the person','D Public order and security offences']}).any(1)
crimedata = crimedata.loc[mask_public]

crimedata.head()

,Year ending March,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,646
1,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,129
2,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,601
3,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,108
4,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",142


In [6]:
crimedata['Offence Division'].value_counts()

A Crimes against the person             74194
D Public order and security offences    34835
Name: Offence Division, dtype: int64

In [7]:
# Sort offenses by Offence Subdivision to identify targeted crimes that are targeted at a person(s)

crimedata['Offence Subdivision'].value_counts()

A20 Assault and related offences                       33221
A70 Stalking, harassment and threatening behaviour     17804
D20 Disorderly and offensive conduct                   16929
D10 Weapons and explosives offences                    11833
A80 Dangerous and negligent acts endangering people    10158
Other crimes against the person                         8378
D30 Public nuisance offences                            5778
A50 Robbery                                             4633
D40 Public security offences                             295
Name: Offence Subdivision, dtype: int64

In [8]:
mask_targeted = ~crimedata.isin({'Offence Subdivision': ['A50 Robbery','Weapons and explosives offences', 'Other crimes against the person']}).any(1)
crimedata = crimedata.loc[mask_targeted]

crimedata.head()

,Year ending March,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,646
1,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,129
2,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,601
3,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,108
4,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",142


In [9]:
crimedata['Offence Subdivision'].value_counts()

A20 Assault and related offences                       33221
A70 Stalking, harassment and threatening behaviour     17804
D20 Disorderly and offensive conduct                   16929
D10 Weapons and explosives offences                    11833
A80 Dangerous and negligent acts endangering people    10158
D30 Public nuisance offences                            5778
D40 Public security offences                             295
Name: Offence Subdivision, dtype: int64

In [10]:
# Sort offenses by Offence Subgroup to identify that are of non domestic in nature

crimedata['Offence Subgroup'].value_counts()

A231 FV Common assault                                                7739
A212 Non-FV Serious assault                                           7654
A211 FV Serious assault                                               7506
A232 Non-FV Common assault                                            6589
D12 Prohibited and controlled weapons offences                        5331
D22 Drunk and disorderly in public                                    5271
D11 Firearms offences                                                 5254
A89 Other dangerous or negligent acts endangering people              5209
D23 Offensive conduct                                                 4513
A732 Non-FV Threatening behaviour                                     4361
A731 FV Threatening behaviour                                         4043
A22 Assault police, emergency services or other authorised officer    3733
D35 Improper movement on public or private space                      3651
A712 Non-FV Stalking     

In [11]:
crimedata[['Offence Subgroup Code', 'Offence Subgroup Name']] = crimedata['Offence Subgroup'].str.split(' ', n=1, expand = True)

In [12]:
crimedata.head()

,Year ending March,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,Offence Subgroup Code,Offence Subgroup Name
0,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,646,A232,Non-FV Common assault
1,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,129,A231,FV Common assault
2,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,601,A212,Non-FV Serious assault
3,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,108,A211,FV Serious assault
4,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",142,A22,"Assault police, emergency services or other au..."


In [13]:
mask_nondomestic = ~crimedata.isin({'Offence Subgroup Code': ['A231', 'A211', 'A731', 'A711', 'A721' ]}).any(1)
crimedata = crimedata.loc[mask_nondomestic]

In [14]:
crimedata['Offence Subgroup'].value_counts()

A212 Non-FV Serious assault                                           7654
A232 Non-FV Common assault                                            6589
D12 Prohibited and controlled weapons offences                        5331
D22 Drunk and disorderly in public                                    5271
D11 Firearms offences                                                 5254
A89 Other dangerous or negligent acts endangering people              5209
D23 Offensive conduct                                                 4513
A732 Non-FV Threatening behaviour                                     4361
A22 Assault police, emergency services or other authorised officer    3733
D35 Improper movement on public or private space                      3651
A712 Non-FV Stalking                                                  3342
A83 Throw or discharge object endangering people                      2791
D24 Offensive language                                                2502
D25 Criminal intent      

In [15]:
# Remove any other offenses that are not relevant

mask_irrelevant = ~crimedata.isin({'Offence Subgroup Code': ['D12', 'D11', 'A22', 'D25', 'A82', 'D33', 'D43', 'D31', 'D44'
                                                            ,'D41', 'D34', 'D42']}).any(1)
crimedata = crimedata.loc[mask_irrelevant]

In [16]:
crimedata['Offence Subgroup'].value_counts()

A212 Non-FV Serious assault                                 7654
A232 Non-FV Common assault                                  6589
D22 Drunk and disorderly in public                          5271
A89 Other dangerous or negligent acts endangering people    5209
D23 Offensive conduct                                       4513
A732 Non-FV Threatening behaviour                           4361
D35 Improper movement on public or private space            3651
A712 Non-FV Stalking                                        3342
A83 Throw or discharge object endangering people            2791
D24 Offensive language                                      2502
A722 Non-FV Harassment and private nuisance                 2075
A81 Dangerous driving                                       1730
D21 Riot and affray                                         1505
D13 Explosives offences                                     1248
D26 Disorderly conduct                                       993
D32 Hoaxes               

# Calculate weights for each crime based on severity

In [17]:
# Function to calculate severity of crime. The severity is out of 10.

def crime_severity (row):
    if row['Offence Subgroup Code'] == 'A212' :
        return 10
    if row['Offence Subgroup Code'] == 'A232' :
        return 10
    if row['Offence Subgroup Code'] == 'D22' :
        return 4
    if row['Offence Subgroup Code'] == 'A89' :
        return 7
    if row['Offence Subgroup Code'] == 'D23' :
        return 5
    if row['Offence Subgroup Code'] == 'A732' :
        return 7
    if row['Offence Subgroup Code'] == 'D35' :
        return 4
    if row['Offence Subgroup Code'] == 'A712' :
        return 10
    if row['Offence Subgroup Code'] == 'A83' :
        return 7
    if row['Offence Subgroup Code'] == 'D24' :
        return 3
    if row['Offence Subgroup Code'] == 'A722' :
        return 8
    if row['Offence Subgroup Code'] == 'A81' :
        return 7
    if row['Offence Subgroup Code'] == 'D21' :
        return 6
    if row['Offence Subgroup Code'] == 'D13' :
        return 6
    if row['Offence Subgroup Code'] == 'D26' :
        return 5
    if row['Offence Subgroup Code'] == 'D32' :
        return 7
    if row['Offence Subgroup Code'] == 'D36' :
        return 7
    if row['Offence Subgroup Code'] == 'D49' :
        return 7

In [18]:
crime_processed = crimedata.copy(deep=True)

crime_processed['Crime Severity'] = crime_processed.apply (lambda row: crime_severity (row),axis=1)

crime_processed.head()

,Year ending March,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,Offence Subgroup Code,Offence Subgroup Name,Crime Severity
0,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,646,A232,Non-FV Common assault,10
2,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,601,A212,Non-FV Serious assault,10
7,2018,3000,MELBOURNE,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A732 Non-FV Threatening behaviour,75,A732,Non-FV Threatening behaviour,7
9,2018,3000,MELBOURNE,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A722 Non-FV Harassment and private nuisance,20,A722,Non-FV Harassment and private nuisance,8
11,2018,3000,MELBOURNE,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A712 Non-FV Stalking,21,A712,Non-FV Stalking,10


In [19]:
# Select most recent crime data from 2018

crime_processed = crime_processed.loc[(crime_processed['Year ending March'] == 2018)]
crime_processed.head()

,Year ending March,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,Offence Subgroup Code,Offence Subgroup Name,Crime Severity
0,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A232 Non-FV Common assault,646,A232,Non-FV Common assault,10
2,2018,3000,MELBOURNE,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,601,A212,Non-FV Serious assault,10
7,2018,3000,MELBOURNE,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A732 Non-FV Threatening behaviour,75,A732,Non-FV Threatening behaviour,7
9,2018,3000,MELBOURNE,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A722 Non-FV Harassment and private nuisance,20,A722,Non-FV Harassment and private nuisance,8
11,2018,3000,MELBOURNE,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",A712 Non-FV Stalking,21,A712,Non-FV Stalking,10


In [20]:
# Function to calculate severity score per postcode. 

def severity_score (row):
    return row['Incidents Recorded'] * row['Crime Severity']

In [21]:
crime_processed['Severity Total Score'] = crime_processed.apply (lambda row: severity_score (row),axis=1)
crime_processed.head()

crime_by_offence = crime_processed.copy(deep=True)


In [22]:
crime_processed['Year ending March'].value_counts()

2018    6215
Name: Year ending March, dtype: int64

In [23]:
crime_processed['Suburb/Town Name'].value_counts()

MELBOURNE           18
MORWELL             18
WERRIBEE            17
FITZROY             17
WARRNAMBOOL         17
FRANKSTON           17
MILDURA             17
HAMILTON            16
ECHUCA              16
ST KILDA            16
DANDENONG           16
CRANBOURNE          16
BENALLA             16
SHEPPARTON          16
TRARALGON           16
PAKENHAM            16
HORSHAM             15
CORIO               15
NORTH MELBOURNE     15
ST ALBANS           15
EPPING              15
NORLANE             15
WODONGA             15
GEELONG             15
NARRE WARREN        15
COLLINGWOOD         15
CRAIGIEBURN         15
SALE                15
CAROLINE SPRINGS    15
WONTHAGGI           15
                    ..
KOOYONG              1
PERCYDALE            1
THE GURDIES          1
BORUNG               1
MERBEIN SOUTH        1
LANGLEY              1
MUCKLEFORD           1
BREWSTER             1
SANDY POINT          1
KATUNGA              1
CHARLEROI            1
BOHO                 1
OXLEY FLATS

In [24]:
crime_processed['Offence Division'].value_counts()

A Crimes against the person             3953
D Public order and security offences    2262
Name: Offence Division, dtype: int64

In [25]:
crime_processed['Offence Subdivision'].value_counts()

A20 Assault and related offences                       1521
D20 Disorderly and offensive conduct                   1507
A80 Dangerous and negligent acts endangering people    1317
A70 Stalking, harassment and threatening behaviour     1115
D30 Public nuisance offences                            618
D10 Weapons and explosives offences                     133
D40 Public security offences                              4
Name: Offence Subdivision, dtype: int64

In [26]:
crime_processed['Incidents Recorded'].value_counts()

1       2681
2       1046
3        506
4        381
5        279
6        189
9        119
7        119
8        107
10        76
13        61
11        59
12        52
14        43
17        32
16        31
15        27
19        27
20        24
18        22
23        19
22        17
24        16
26        16
21        14
28        13
25        13
30        12
27        11
31         8
        ... 
276        1
36         1
199        1
207        1
232        1
129        1
452        1
139        1
71         1
79         1
646        1
170        1
142        1
126        1
86         1
82         1
78         1
66         1
403        1
107        1
601        1
245        1
145        1
113        1
105        1
77         1
69         1
115        1
135        1
1343       1
Name: Incidents Recorded, Length: 110, dtype: int64

In [27]:
crime_processed['Offence Subgroup Code'].value_counts()

A212    782
A232    739
A89     638
D22     569
A732    486
D35     446
D23     426
A81     368
A712    343
A83     311
A722    286
D24     254
D21     164
D13     133
D32      98
D26      94
D36      74
D49       4
Name: Offence Subgroup Code, dtype: int64

In [28]:
crime_processed['Offence Subgroup Name'].value_counts()

Non-FV Serious assault                                  782
Non-FV Common assault                                   739
Other dangerous or negligent acts endangering people    638
Drunk and disorderly in public                          569
Non-FV Threatening behaviour                            486
Improper movement on public or private space            446
Offensive conduct                                       426
Dangerous driving                                       368
Non-FV Stalking                                         343
Throw or discharge object endangering people            311
Non-FV Harassment and private nuisance                  286
Offensive language                                      254
Riot and affray                                         164
Explosives offences                                     133
Hoaxes                                                   98
Disorderly conduct                                       94
Other public nuisance offences          

In [29]:
crime_processed['Crime Severity'].value_counts()


7     1979
10    1864
4     1015
5      520
6      297
8      286
3      254
Name: Crime Severity, dtype: int64

In [30]:
crime_processed.drop(['Year ending March','Suburb/Town Name','Offence Division','Offence Subdivision',
                      'Incidents Recorded','Offence Subgroup Code','Offence Subgroup Name','Crime Severity'], axis=1, inplace=True, errors='raise')

In [31]:
crime_processed = crime_processed.groupby(['Postcode'], as_index=False).agg({'Severity Total Score':sum})

In [32]:
crime_processed.head()

,Postcode,Severity Total Score
0,3000,22498
1,3002,681
2,3003,524
3,3006,3266
4,3008,2058


In [33]:
crime_processed[crime_processed.duplicated(keep=False)].sort_values(by=['Postcode'])

,Postcode,Severity Total Score


# Join with postcodes to get average gps position of each postcode

In [34]:
# Read postcode data
postcode_loc = pd.read_csv('Australian_Post_Codes_Lat_Lon.csv')
postcode_loc.head()

,postcode,suburb,state,dc,type,lat,lon
0,200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,AUSTRALIAN NATIONAL UNI LPO,Post Office Boxes,-35.277272,149.117136
1,221,BARTON,ACT,NaN,LVR,-35.201372,149.095065
2,800,DARWIN,NT,DARWIN DELIVERY CENTRE,Delivery Area,-12.801028,130.955789
3,801,DARWIN,NT,DARWIN DELIVERY CENTRE,Post Office Boxes,-12.801028,130.955789
4,804,PARAP,NT,PARAP,Post Office Boxes,-12.432181,130.843310


In [35]:
postcode_loc.shape

(16080, 7)

In [36]:
postcode_loc = postcode_loc.groupby(['postcode'], as_index=False).mean()
postcode_loc.head()

,postcode,lat,lon
0,0,NaN,NaN
1,200,-35.277272,149.117136
2,221,-35.201372,149.095065
3,800,-12.801028,130.955789
4,801,-12.801028,130.955789


In [37]:
postcode_loc[postcode_loc.duplicated(keep=False)].sort_values(by=['postcode'])

,postcode,lat,lon


In [38]:
crime_processed = crime_processed.set_index('Postcode').join(postcode_loc.set_index('postcode'))

In [39]:
crime_processed.reset_index()

,Postcode,Severity Total Score,lat,lon
0,3000,22498,-37.814563,144.970267
1,3002,681,-37.816640,144.987811
2,3003,524,-37.806255,144.941123
3,3006,3266,-37.823258,144.965926
4,3008,2058,-37.814719,144.948039
5,3011,2246,-37.801199,144.887090
6,3012,729,-37.800197,144.867860
7,3013,359,-37.817099,144.886678
8,3015,496,-37.835258,144.879655
9,3016,675,-37.857292,144.892369


In [40]:
crime_processed.to_csv('crimes_weighted.csv', index=True)

In [41]:
crime_by_offence.to_csv('crimes_by_offence.csv', index=True)

# Import and Run K Nearest Neighbour to Classify tram stops with a score

In [42]:
from sklearn import preprocessing, cross_validation, neighbors

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [43]:
tram_data = pd.read_csv('stop_locations.txt', sep="|",names = ["StopLocationID","StopNameShort","StopNameLong","StopType","SuburbName"
                                                               ,"PostCode","RegionName", "LocalGovernmentArea","StatDivision","lat","lon"] )
tram_data.head()

,StopLocationID,StopNameShort,StopNameLong,StopType,SuburbName,PostCode,RegionName,LocalGovernmentArea,StatDivision,lat,lon
0,867,Weemala Court,Weemala Ct/Plenty River Dr (Greensborough),Kerbside,Greensborough,3088.0,Melbourne,Banyule,Greater Metro,-37.689596,145.105088
1,868,Crana Grove,Crana Gr/Plenty River Dr (Greensborough),Kerbside,Greensborough,3088.0,Melbourne,Banyule,Greater Metro,-37.686742,145.105588
2,869,Punkerri Circuit,Punkerri Cct/Plenty River Dr (Greensborough),Kerbside,Greensborough,3088.0,Melbourne,Banyule,Greater Metro,-37.683643,145.108743
3,870,Plenty River Drive,231 Plenty River Dr (Greensborough),Kerbside,Greensborough,3088.0,Melbourne,Banyule,Greater Metro,-37.682591,145.111331
4,875,Oldstead Rd,Oldstead Rd/Diamond Creek Rd (Greensborough),Kerbside,Greensborough,3088.0,Melbourne,Banyule,Greater Metro,-37.685336,145.117319


In [44]:
tram_data_processed = tram_data[['StopNameShort','lat', 'lon']].copy()
tram_data_processed.head()

,StopNameShort,lat,lon
0,Weemala Court,-37.689596,145.105088
1,Crana Grove,-37.686742,145.105588
2,Punkerri Circuit,-37.683643,145.108743
3,Plenty River Drive,-37.682591,145.111331
4,Oldstead Rd,-37.685336,145.117319


In [45]:
tram_model = tram_data_processed[['lat', 'lon']].copy()
tram_model.head()

,lat,lon
0,-37.689596,145.105088
1,-37.686742,145.105588
2,-37.683643,145.108743
3,-37.682591,145.111331
4,-37.685336,145.117319


In [46]:
tram_model.shape

(27614, 2)

In [47]:
# Run KNN

In [48]:
crime_model = crime_processed[['lat', 'lon','Severity Total Score']].copy()
crime_model = crime_model.reset_index(drop=True)
crime_model.head()

,lat,lon,Severity Total Score
0,-37.814563,144.970267,22498
1,-37.816640,144.987811,681
2,-37.806255,144.941123,524
3,-37.823258,144.965926,3266
4,-37.814719,144.948039,2058


In [49]:
# Train - Test split

In [50]:
crime_model[crime_model.isnull().any(axis=1)]

,lat,lon,Severity Total Score
183,NaN,NaN,28
203,NaN,NaN,21
261,NaN,NaN,7
293,NaN,NaN,10
432,NaN,NaN,4
471,NaN,NaN,24
576,NaN,NaN,214


In [51]:
crime_model = crime_model.dropna()

In [52]:
X = np.array(crime_model.drop(['Severity Total Score'],1))
y = np.array(crime_model['Severity Total Score'])

In [53]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2)

In [54]:
# Train the model
clf = neighbors.KNeighborsClassifier(n_neighbors=1, weights = 'distance', p=2)
clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='distance')

In [55]:
#Note that this is not a real accuracy so ignore
accuracy = clf.score(X_test,y_test)
print(accuracy)

0.008333333333333333


In [56]:
# Predict nearest neighbour of tram data

tram_model_x = np.array(tram_model)
tram_model_x

array([[-37.689596, 145.105088],
       [-37.686742, 145.105588],
       [-37.683643, 145.108743],
       ...,
       [-38.060197, 145.409172],
       [-38.059545, 145.405684],
       [-38.056718, 145.390294]])

In [57]:
# Predict severity score of each stop

tram_model['ComputedSeverityScore'] = clf.predict(tram_model_x)

In [58]:
tram_model['ComputedSeverityScore']

0          98
1          98
2          98
3          98
4          98
5          98
6        1083
7         981
8        1083
9        1083
10       1083
11        834
12        834
13        834
14        834
15        834
16        834
17        834
18        834
19        834
20        834
21         52
22         52
23         52
24         52
25         52
26         52
27         52
28         52
29         52
         ... 
27584     843
27585     758
27586    1367
27587      74
27588     523
27589     523
27590     360
27591    1829
27592    5395
27593    1367
27594    1043
27595      93
27596     282
27597     282
27598     282
27599     282
27600     282
27601     282
27602     282
27603     282
27604     282
27605     282
27606     282
27607     282
27608     282
27609     282
27610     282
27611     282
27612     282
27613     241
Name: ComputedSeverityScore, Length: 27614, dtype: int64

In [59]:
tram_model['Weight'] = tram_model['ComputedSeverityScore']/tram_model['ComputedSeverityScore'].max().astype(np.float64)


In [60]:
tram_model = tram_model.drop(['ComputedSeverityScore'], axis=1)
tram_model.columns = ['Latitude', 'Longitude', 'Weight']

In [61]:
tram_model.head()

,Latitude,Longitude,Weight
0,-37.689596,145.105088,0.004356
1,-37.686742,145.105588,0.004356
2,-37.683643,145.108743,0.004356
3,-37.682591,145.111331,0.004356
4,-37.685336,145.117319,0.004356


In [62]:
tram_model.to_csv('tram_model.csv', index=True)